<a href="https://colab.research.google.com/github/Aynahd/VeriAssist/blob/main/bonusAttempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data re-structuring for CNN
1D to 3 by 2 matrix

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('drug200.csv')
def encode_column(column):
    unique_values = column.unique()
    mapping = {value: idx for idx, value in enumerate(unique_values)}
    return column.map(mapping), mapping

def normalize_column(column):
    return (column - column.min()) / (column.max() - column.min())

df['Sex'], sex_mapping = encode_column(df['Sex'])
df['BP'], bp_mapping = encode_column(df['BP'])
df['Cholesterol'], cholesterol_mapping = encode_column(df['Cholesterol'])
df['Drug'], drug_mapping = encode_column(df['Drug'])


df['Age'] = normalize_column(df['Age'])
df['Na_to_K'] = normalize_column(df['Na_to_K'])
print(df)

          Age  Sex  BP  Cholesterol   Na_to_K  Drug
0    0.135593    0   0            0  0.596848     0
1    0.542373    1   1            0  0.213397     1
2    0.542373    1   1            0  0.120239     1
3    0.220339    0   2            0  0.047814     2
4    0.779661    0   1            0  0.368191     0
..        ...  ...  ..          ...       ...   ...
195  0.694915    0   1            0  0.165676     1
196  0.016949    1   1            0  0.179405     1
197  0.627119    1   2            0  0.113359     2
198  0.135593    1   2            1  0.242385     2
199  0.423729    0   1            1  0.158859     2

[200 rows x 6 columns]


In [7]:
y = torch.tensor(df['Drug'].values, dtype=torch.long)
print(y)

tensor([0, 1, 1, 2, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 2, 3, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 0, 0, 2, 4, 2, 0, 2, 2, 3, 2, 2, 2, 0, 4, 0, 2, 2, 2, 3, 1,
        0, 0, 0, 2, 0, 0, 4, 1, 4, 0, 2, 0, 0, 3, 0, 2, 4, 0, 3, 2, 0, 0, 4, 0,
        2, 0, 0, 0, 3, 0, 3, 2, 4, 2, 1, 3, 1, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 0, 0, 3, 3, 1, 2, 0, 2, 2, 0, 4, 0, 3, 2, 2, 2, 2, 0, 2, 2, 3, 0,
        0, 0, 0, 0, 4, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 4, 3, 4, 2, 3, 0, 4, 0,
        3, 2, 2, 3, 2, 1, 3, 4, 2, 2, 0, 1, 3, 0, 1, 2, 2, 4, 2, 0, 0, 0, 0, 2,
        0, 3, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 0, 2, 2, 0, 0, 0, 4, 3, 0, 0, 0, 3,
        0, 1, 0, 1, 1, 2, 2, 2])


Restruct

In [9]:
X = df.iloc[:, :-1].values
dummy_column = np.zeros((X.shape[0], 1))
X_extended = np.hstack((X, dummy_column))

X_cnn = X_extended.reshape(X_extended.shape[0], 1, 3, 2)

print("Reshaped X for CNN:", X_cnn.shape)
print("Target y shape:", y.shape)

Reshaped X for CNN: (200, 1, 3, 2)
Target y shape: torch.Size([200])


In [10]:
print(X_cnn)

[[[[0.13559322 0.        ]
   [0.         0.        ]
   [0.59684783 0.        ]]]


 [[[0.54237288 1.        ]
   [1.         0.        ]
   [0.21339671 0.        ]]]


 [[[0.54237288 1.        ]
   [1.         0.        ]
   [0.12023891 0.        ]]]


 ...


 [[[0.62711864 1.        ]
   [2.         0.        ]
   [0.11335918 0.        ]]]


 [[[0.13559322 1.        ]
   [2.         1.        ]
   [0.24238539 0.        ]]]


 [[[0.42372881 0.        ]
   [1.         1.        ]
   [0.15885922 0.        ]]]]


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np

In [14]:
X = torch.tensor(X.reshape(200, 1, 3, 2), dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

ValueError: cannot reshape array of size 1000 into shape (200,1,3,2)

In [ ]:
dataset = TensorDataset(X, y)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=2)  # 1 input channel, 32 output channels
        self.pool = nn.MaxPool2d(2, 2)               # Pooling layer
        self.fc1 = nn.Linear(32, 64)
        self.fc2 = nn.Linear(64, num_classes)        # Output layer

    def forward(self, x):
        x = torch.relu(self.conv1(x))  # Apply convolution + ReLU activation
        x = self.pool(x)               # Apply pooling
        x = x.view(x.size(0), -1)      # Flatten for the fully connected layer
        x = torch.relu(self.fc1(x))    # First fully connected layer
        x = self.fc2(x)                # Output layer (no activation for logits)
        return x
              # Fully connected layer

num_classes = len(torch.unique(y))  # Number of classes in y
model = CNN(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()           # Zero gradients
        outputs = model(inputs)         # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()                 # Backward pass
        optimizer.step()                # Update weights
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():  # No need to compute gradients during evaluation
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get class predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f"Test Accuracy: {accuracy:.2f}%")